In [1]:
import cv2
import cv2 as cv
import imutils
import os
from os import path
import numpy as np
import torch
import matplotlib.pyplot as plt
import pickle
import torchvision.transforms.functional as F
import copy
# Add code to sys.path
import matplotlib.pyplot as plt 

import sys
# sys.path.append('../src')

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import cv2 as cv
import collections

import dla.src.table_structure_analysis as tsa
import dla.src.xml_utils as xml_utils
from dla.src.image_utils import put_box, put_line
import pytesseract
import numpy as np

from statistics import mean

import importlib
import glosat_utils

from glosat_utils import *
from PIL import Image

In [ ]:
def get_table_coord(cell, tables):
    for idx,table in enumerate(tables):
        table = list(map(int, table))
        score = tsa.how_much_contained(cell[0:4],table)    
        if score > 0.5:
            return idx, score 
    return [], 0


In [12]:
def find_average_cellsize(tables, copy_cells, THRESHOLD=0.5, init=False, average_cordinates = []):
    #Separate cells for each tables
    table_cells = [[] for i in range(len(tables))]
    cell_size= [[[],[]] for i in range(len(tables))]
    xy_coords = [[[],[]] for i in range(len(tables))]
    cells = []
    if init:
        for cell in copy_cells:
            if cell[4] > THRESHOLD:
                table_idx, score = get_table_coord(cell, tables)
                if score > 0.5:
                    cell[4] = table_idx  #cells and its table index
                    if isinstance(table_idx, int):
                        table_cells[table_idx].append(cell)
                        cells.append(cell)
    else:
        for cell in copy_cells:
            table_idx = cell[4]
            if isinstance(table_idx, int):
                table_cells[table_idx].append(cell)
        
    # find average cell size
    for tid, tab_cells in enumerate(table_cells):
        # print('average_cordinates',len(average_cordinates))
        if len(tab_cells)>0:
            x_size = []
            y_size = []
            for cell in tab_cells:
                xax = abs(cell[0]-cell[2])
                yax = abs(cell[1]-cell[3])
                xy_coords[tid][0]+=[cell[0],cell[2]]
                xy_coords[tid][1]+=[cell[1],cell[3]]
                x_size.append(xax)
                y_size.append(yax)
            x_avg = mean(x_size)
            y_avg = mean(y_size)
            average_cordinates.append((x_avg, y_avg))
            xy_coords[tid][0] = sort_coordinates(xy_coords[tid][0]+[tables[tid][0],tables[tid][2]])
            xy_coords[tid][1] = sort_coordinates(xy_coords[tid][1]+[tables[tid][1],tables[tid][3]]) 
            cell_size[tid][0] = sort_coordinates(x_size) 
            cell_size[tid][1] = sort_coordinates(y_size) 
        else:
            average_cordinates.append((0, 0))
        
    # Normalize coordinates
    for tid,tab_cells in enumerate(table_cells):
        cells_x, cells_y = xy_coords[tid]
        x_size, y_size = cell_size[tid]
        if len(cell_size[tid][0])>0 and len(cell_size[tid][1])>0:
            x_avg, y_avg = average_cordinates[tid] 
            cell_size_min = cell_size[tid][0][0]
            xy_coords[tid][0] = remove_small_coordinates(xy_coords[tid][0], cell_size_min)
            cell_size_min = cell_size[tid][1][0]
            xy_coords[tid][1] = remove_small_coordinates(xy_coords[tid][1], cell_size_min)
            
    return average_cordinates, cell_size, xy_coords, cells, table_cells


In [40]:
def find_overlapped_cell_final_v2(corrected_cells, cells_pred, average_cordinates,pc=0.5):
    new_cells_pred = []
    large_cells = []
    missed_cells = []
    overlapped = copy.deepcopy(corrected_cells)
    
    for i, box in enumerate(cells_pred):
        count=0       
        xi = abs(box[2]-box[0])
        yi = abs(box[3]-box[1])
        tid = box[4]
        if isinstance(tid, int):
            avg_x, avg_y = average_cordinates[tid]
            
            if avg_x>0 and avg_y>0:
                if xi>(avg_x*pc*3) and yi>(avg_y*pc*3):        
                    for j, cells in enumerate(corrected_cells): 
                        xj = abs(cells[2]-cells[0])
                        yj = abs(cells[3]-cells[1])

                        if xj>(avg_x*pc*3) and yj>(avg_y*pc*3):   
                            if tsa.how_much_contained(cells,box)>pc:
                                count+=1
    #                             if xj < xi:
    #                                 cells[0] = box[0]  
    #                                 cells[2] = box[2]
    #                             if yj < yi:
    #                                 cells[1] = box[1]  
    #                                 cells[3] = box[3]
                                overlapped[j] = cells
                                break

                            elif tsa.how_much_contained(box, cells)>pc:
                                count+=1
    #                             if xj < xi:
    #                                 cells[0] = box[0]  
    #                                 cells[2] = box[2]
    #                             if yj < yi:
    #                                 cells[1] = box[1]  
    #                                 cells[3] = box[3]
                                overlapped[j] = box
                                break
                            else:
                                missed_cells.append(box)
                        else:
                            large_cells.append(box)
                else:
                    large_cells.append(box)
                        
            if count==0:
                new_cells_pred.append(box)
    seen = set()
    unique_lists = [x for x in missed_cells if tuple(x) not in seen and not seen.add(tuple(x))]

    return new_cells_pred, overlapped, large_cells,unique_lists

In [ ]:
config_file = '/home/gyanendro/Desktop/active_learning-2/mmdetection/dla/config/cascadeRCNN_ignore_all_but_cells.py'
table_checkpoint_file = '/home/gyanendro/Desktop/active_learning-2/dla_models/model_semi_supervised_fine_aclr_0/table_det/model_fulltables_only_GloSAT.pth'
table_hearder_checkpoint_file = '/home/gyanendro/Desktop/active_learning-2/dla_models/model_semi_supervised_fine_aclr_0/table_det/model_tables_enchanced_GloSAT.pth'
coarse_cell_checkpoint_file = f'/home/gyanendro/Desktop/active_learning-2/dla_models/model_semi_supervised_fine_aclr_0/coarse_cell_det/best_model_manual.pth'
cell_checkpoint_file_dir = f'/home/gyanendro/Desktop/active_learning-2/dla_models/model_semi_supervised_fine_aclr_0/supervised-checkpoint' #best_model_aclr_0_epc_601.pth'
# files = os.listdir(directory)

In [52]:
cell_cordinates_info = {}
img_path = '/home/gyanendro/Desktop/mm-ocr-update/data/African-American-France/Images/Bear-AG-29-1940-09_10.jpg'

file = img_path.split('/')[-1]
filename = file.split('.')[0]

num_aclr = 12 # Number of ensembled models

outdirectory = '/home/gyanendro/Desktop/mm-ocr-update/Tabular-data-extraction/American'

In [53]:
if not os.path.exists(outdirectory):
    os.mkdir(outdirectory)

outpathXML = f'{outdirectory}/XML-det'
if not os.path.exists(outpathXML):
    os.mkdir(outpathXML)
    
outpathJSON = f'{outdirectory}/JSON-det'
if not os.path.exists(outpathJSON):
    os.mkdir(outpathJSON)
    

In [54]:
model = init_detector(config_file, table_checkpoint_file, device='cuda:0')
tb_header_model = init_detector(config_file, table_hearder_checkpoint_file, device='cuda:0')
coarse_cell_model = init_detector(config_file, coarse_cell_checkpoint_file, device='cuda:0')

THRESHOLD = 0.5
CLASSES = ("table_body","cell","full_table","header","heading")
color = 255



/home/gyanendro/Desktop/active_learning-2/mmdetection/mmdet/apis/inference.py:40: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [55]:
print(f'Load image: {img_path}')
img = cv.imread(img_path)
# Table detection
result = inference_detector(model,img)
result_header = inference_detector(tb_header_model,img)

#Process table headers
headers = []
for box in result_header[CLASSES.index("header")]:
    if box[4]>THRESHOLD :
        headers.append(box[0:4])


#Process table headers
tables_body = []
for box in result_header[CLASSES.index("table_body")]:
    if box[4]>THRESHOLD :
        tables_body.append(box[0:4])


#Process table bodies
tables = []
for box in result_header[CLASSES.index("table_body")]:
    if box[4]>THRESHOLD :
        tables.append(box[0:4])

#Process tables
full_tables = []
for box in result_header[CLASSES.index("full_table")]:
    if box[4]>THRESHOLD :
        full_tables.append(box[0:4])

        if all(tsa.how_much_contained(table,box)<0.5 for table in tables):
            tables.append(box[0:4])
            

Load image: /home/gyanendro/Desktop/mm-ocr-update/data/African-American-France/Images/Bear-AG-29-1940-09_10.jpg


In [56]:
full_tables, tables

([array([1161.3949, 2119.5186, 9141.568 , 5852.644 ], dtype=float32),
  array([ 1131.82 ,  9839.624,  9158.682, 12148.756], dtype=float32)],
 [array([1166.721 , 3440.9912, 9155.126 , 5843.133 ], dtype=float32),
  array([ 1181.578 , 10306.653 ,  9115.692 , 12209.5625], dtype=float32)])

In [57]:
coarse_cells_by_image = detect_cell_pretrained(coarse_cell_model,img)
cells_by_image = collections.defaultdict(list)
for aclr in reversed(range(int(num_aclr)-2,int(num_aclr)+1)): 
    cell_checkpoint_file = f'{cell_checkpoint_file_dir}/aclr{aclr}.pth'
    # cell_checkpoint_file = f'{cell_checkpoint_file_dir}/best_epoch_200.pth'
    cell_model = init_detector(config_file, cell_checkpoint_file, device='cuda:0')
    cells_by_image[aclr] = detect_cell_pretrained(cell_model,img)

In [58]:
cells_ori = []
cells_ori_missed = []
idx_cell = []

for aclr in reversed(range(int(num_aclr)-2,int(num_aclr)+1)): 
    idx_cell = []
    for cell in cells_by_image[aclr]:
        idx, score = get_table_coord(cell, tables)
        print(cell, tables, score)
        if isinstance(idx, int):
            cell[4] =  idx
            idx_cell.append(cell)
            
    table_cells = get_x_y_initial(tables, cells_by_image[aclr], THRESHOLD)
    cell_cordinates, average_cordinates = get_coordinates(table_cells,0.5)

    not_overlapped, overlapped, large_cell, missed = find_overlapped_cell_final_v2(cells_ori, idx_cell, average_cordinates,pc=0.001)
    cells_ori+=not_overlapped
    not_overlapped, overlapped, large_cell, missed = find_overlapped_cell_final_v2(cells_ori_missed, missed, average_cordinates,pc=0.005)
    cells_ori_missed+=not_overlapped

[6431.30908203125, 3903.725830078125, 7079.712890625, 4072.403564453125, 0.9990045428276062] [array([1166.721 , 3440.9912, 9155.126 , 5843.133 ], dtype=float32), array([ 1181.578 , 10306.653 ,  9115.692 , 12209.5625], dtype=float32)] 1.0
[6431.87841796875, 4228.54345703125, 7091.7900390625, 4388.4736328125, 0.9988672733306885] [array([1166.721 , 3440.9912, 9155.126 , 5843.133 ], dtype=float32), array([ 1181.578 , 10306.653 ,  9115.692 , 12209.5625], dtype=float32)] 1.0
[7591.44970703125, 4072.162109375, 7941.21337890625, 4236.40771484375, 0.9988638162612915] [array([1166.721 , 3440.9912, 9155.126 , 5843.133 ], dtype=float32), array([ 1181.578 , 10306.653 ,  9115.692 , 12209.5625], dtype=float32)] 1.0
[6430.16455078125, 3633.063720703125, 7097.39306640625, 3910.739501953125, 0.9987842440605164] [array([1166.721 , 3440.9912, 9155.126 , 5843.133 ], dtype=float32), array([ 1181.578 , 10306.653 ,  9115.692 , 12209.5625], dtype=float32)] 1.0
[7590.91943359375, 4386.4296875, 7933.876953125, 4

In [ ]:
image = cv.imread(img_path)
img, height, width, _ = image_preprocessing(img)

color = 255
for box in tables:
    box = list(map(int, box[0:4]))
    put_box(image,box,(0,color,color)) # Cyan 
    
for box in headers:
    box = list(map(int, box[0:4]))
    put_box(image,box,(color,0,color), 'Header') # magenta 
    
for box in list(cells_by_image[aclr]):
    put_box(image,box,(0,0,color)) # Blue
    
    
# for box in list(new_col_cells):
#     put_box(image,box,(0,color,color)) # cyan
    
im_pil = Image.fromarray(image)
im_pil

In [49]:
(tables, cells_by_image[aclr], THRESHOLD)

([array([1296.3898, 3494.409 , 8988.737 , 5612.9595], dtype=float32)],
 [[19093.56640625,
   4564.54443359375,
   19547.091796875,
   4970.400390625,
   0.9990782737731934],
  [21075.595703125,
   11183.9169921875,
   21576.189453125,
   11536.3876953125,
   0.9981845021247864],
  [19531.99609375,
   4598.998046875,
   19899.005859375,
   5061.50048828125,
   0.995058536529541],
  [21579.78125,
   10628.201171875,
   22105.486328125,
   10983.7568359375,
   0.9948813915252686],
  [19928.576171875,
   4680.2021484375,
   20391.15234375,
   5212.1572265625,
   0.9912142753601074],
  [21013.771484375,
   10790.6904296875,
   21556.26953125,
   11157.5498046875,
   0.9898091554641724],
  [19089.08203125,
   2657.143798828125,
   19550.365234375,
   3235.13525390625,
   0.9889267683029175],
  [19534.958984375,
   4430.2275390625,
   19889.638671875,
   4913.64111328125,
   0.9883257150650024],
  [19931.40234375,
   4857.07177734375,
   20378.111328125,
   5380.3623046875,
   0.9875317215919

In [41]:
 not_overlapped, overlapped, large_cell, missed = find_overlapped_cell_final_v2(cells_ori, idx_cell, average_cordinates,pc=0.001)

IndexError: list index out of range

In [ ]:
for cell in coarse_cells_by_image:
    idx, score = get_table_coord(cell, tables)
    cell[4] = idx
    
not_overlapped, overlapped, large_cell, missed = find_overlapped_cell_final_v2(cells_ori, cells_ori_missed, average_cordinates,pc=0.005)

In [14]:
average_cordinates, cell_sizes, xy_coords, cells, tablewise_cells = find_average_cellsize(full_tables, cells_ori+not_overlapped+coarse_cells_by_image, [])

In [22]:
cells_ori

[]

In [23]:
idx_cell

[]